# PageRank

<h3>Autora: Dandara Sousa </h3>

Essa é a resolução da segunda parte do segundo lab da disciplina de Recuperação da Informação. Nessa atividade a ideia é exercitar o algoritmo de PageRank, implementando-o e aplicando num conjunto de dados que representam links entre investidores de bitcoins. Após isso, o que se deve fazer é uma visualização dos resultados com a ferramente Gephi. Abaixo, segue as perguntas que devem ser respondidas nesta atividade:
1. Quantas iterações o PageRank precisou rodar até atingir convergência?
2. Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?
3. Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?